In [ ]:
#install required libraries if needed
!pip3 install transformers==4.33.2
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name_or_path = 'TheBloke/Llama-2-13B-chat-GPTQ'
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, revision='gptq-4bit-32g-actorder_True',device_map='auto',trust_remote_code=False)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
#define the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
start_inst, end_inst = '[INST]', '[/INST]'
start_sys, end_sys = '<<SYS>>\n', '\n<</SYS>>\n\n'

sys_prompt = """\
You are designated as an assistant that identify and extract high-level topics from articles. \
You should avoid giving specific details and provide unique topics solely. Each topic should be represented as a single word\\
"""
#return the concatenation of system prompt and instruction (with corresponding start and end tokens)
def create_prompt(instruction, system_prompt, start_inst=start_inst, end_inst=end_inst, start_sys=start_sys, end_sys=end_sys):
    prompt = start_inst+start_sys+system_prompt+end_sys+instruction+end_inst
    return prompt

In [ ]:
#remove the input prompt from the output
def format_output(raw_out, prompt, tokenizer):
    out = tokenizer.decode(raw_out[0],skip_special_tokens=True)
    out_new = out.replace(prompt,' ')
    final_out = out_new.strip()
    return final_out

In [ ]:
#generate prompt
def generate(instruction, system_prompt, model=model, tokenizer=tokenizer, device=device):
    prompt = create_prompt(instruction,system_prompt)
    encoded_input = tokenizer(prompt, return_tensors="pt").to(device=model.device)
    output = model.generate(input_ids = encoded_input['input_ids'], do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512, temperature=0.7,)
    f = format_output(output,prompt,tokenizer)
    return f

In [ ]:
#it as an example of instruction with 4 provided demonstrations of sentences with topics and a short text for which the model has to define a topic.
inst = "Political parties play a crucial role in modern democracies, serving as a bridge between the government and the public. // Politics \
The economy plays a crucial role in shaping the prosperity and stability of a nation, influencing everything from employment rates to the standard of living. //Economy \
Vegetables are an essential component of a healthy diet, providing a wide range of nutrients that are vital for maintaining good health. // Vegetables \
Football foster teamwork, discipline, and physical fitness, bringing people together across cultures and creating a sense of community and excitement. // Sports \
Fruits are a delicious and nutritious part of a healthy diet, packed with essential vitamins, minerals, and antioxidants that support overall health and well-being. // "

result = generate(inst,sys_prompt)
print(result)

Sure, I can help you with that! Here are the high-level topics I've identified from the given text, represented as single words:

1. Politics
2. Economy
3. Vegetables
4. Sports
5. Fruits

I've avoided giving specific details and focused on the main themes and topics present in the text.
